In [9]:
import random
from collections import namedtuple
from smac.env import StarCraft2Env

import torch.nn as nn
import numpy as np
import os
import torch.nn.functional as F

In [10]:
SC2_PATH = 'D:\Distrib\Kirill\Programs\StarCraft II'
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

random.seed(42)
np.random.seed(42)
os.environ['SC2PATH'] = SC2_PATH

In [3]:
env = StarCraft2Env(map_name="2m2zFOX", difficulty="1", seed=42)

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class AgentDQN(nn.Module):
    def __init__(self, num_states, num_actions):
        super(AgentDQN, self).__init__()
        hidden_layer_num = (num_states + num_actions) // 2
        self.layer_1 = nn.Linear(num_states, hidden_layer_num)
        self.dropout = nn.Dropout(0.1)
        self.layer_2 = nn.Linear(num_states, num_actions)


    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = self.dropout(x)
        out = self.layer_2(x)
        return out

In [ ]:
policy_net = AgentDQN(screen_height, screen_width, n_actions).to(device)
target_net = AgentDQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
